**Check GPU Availability and Setup Home Directory**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('')

In [ ]:
!nvidia-smi

Wed Apr 30 13:24:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
!pip install -q git+https://github.com/sunsmarterjie/yolov12.git roboflow supervision flash-attn

**Download dataset from Roboflow (Roof_type_OR_Roof_Materials)**

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="Your API Key") #enter your own
project = rf.workspace("").project("")
version = project.version(1)
dataset = version.download("yolov8")

**Train_Yolo**

In [ ]:
from ultralytics import YOLO

In [ ]:
from ultralytics import YOLO

#object segmentation

model = YOLO('yolov8m-seg.pt')

results = model.train(data=f'{dataset.location}/data.yaml', epochs=100, batch=8)

**Save the best Performance Matrices**

**Load Best Model for Inference**

In [ ]:
model = YOLO(f'/{HOME}/runs/segment/train/weights/best.pt')




#/content/drive/MyDrive/NEWRMATAPR14/runs/segment/train7/weights/best.pt

In [ ]:
import os
print(f"Model saved at: {os.path.expanduser('~')}/runs/segment/train/weights/best.pt")


**Single Image TIle Inference_Check**

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
from shapely.geometry import Polygon
import os

# --- PARAMETERS ---
MODEL_PATH = f'/{HOME}/runs//best.pt'
IMAGE_PATH = ''
SIMPLIFY_TOLERANCE = 2.0  # adjust to prune more points

# --- LOAD MODEL & IMAGE ---
model = YOLO(MODEL_PATH)
image = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
h, w = image.shape[:2]

# --- RUN INFERENCE & SHOW (optional) ---
results = model(image_rgb)[0]
plt.imshow(results.plot())
plt.axis("off")
plt.title("")
plt.show()

# --- PREPARE OUTPUT FILE ---
base = os.path.splitext(os.path.basename(IMAGE_PATH))[0]
out_txt = os.path.join('' + '.txt')

with open(out_txt, 'w') as f:
    if results.masks is None:
        print("No masks found.")
    else:
        for i, mask_xy in enumerate(results.masks.xy):
            # simplify polygon
            poly = Polygon(mask_xy)
            poly_simple = poly.simplify(SIMPLIFY_TOLERANCE, preserve_topology=False)
            coords = list(poly_simple.exterior.coords)[:-1]  # drop closing point

            # normalize & flatten
            norm = []
            for x, y in coords:
                norm.extend([x / w, y / h])

            # class ID
            cls_id = int(results.boxes.cls[i].item())
            seg_line = f"{cls_id} " + " ".join(f"{c:.6f}" for c in norm)

            # write to file
            f.write(seg_line + "\n")

print(f"Saved simplified labels to: {out_txt}")

**Infer Multiple Image tiles and get the labels in .txt format to create shape files**

In [ ]:
import os
from ultralytics import YOLO
import cv2



# Define paths
import os

# Define base path
base_path = 'r'

# Define the images path (optional, as you're not appending anything here)
images_path = os.path.join(base_path, '')  # This just equals base_path

# Create a folder named 'labels' under base_path
output_dir = os.path.join(base_path, 'labels')
os.makedirs(output_dir, exist_ok=True)

print(f"Label directory created at: {output_dir}")

# Image size
img_w, img_h = 640, 640

# Get all image files
image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

for img_name in image_files:
    img_path = os.path.join(images_path, img_name)
    image = cv2.imread(img_path)
    if image is None:
        print(f"⚠️ Could not load image: {img_path}")
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model(image_rgb)[0]

    label_name = os.path.splitext(img_name)[0] + ".txt"
    label_path = os.path.join(output_dir, label_name)

    roboflow_labels = []
    if results.masks is not None:
        for i, polygon in enumerate(results.masks.xy):
            class_id = int(results.boxes.cls[i].item())
            coords = []
            for x, y in polygon:
                x_norm = round(x / img_w, 8)
                y_norm = round(y / img_h, 8)
                coords.append(f"{x_norm} {y_norm}")
            label_line = f"{class_id} " + " ".join(coords)
            roboflow_labels.append(label_line)

        with open(label_path, 'w') as f:
            for line in roboflow_labels:
                f.write(line + '\n')

        print(f"✅ Saved: {label_path}")
    else:
        print(f"⚠️ No masks for: {img_name}")




**Create .shp files for GIS**

In [ ]:
# === Install Libraries (if running in Colab) ===
!pip install geopandas shapely fiona

# === Import Libraries ===
import os
import geopandas as gpd
from shapely.geometry import Polygon

# === Editable Paths ===
pgw_dir = ""      # Folder containing PGW files
labels_dir = ""   # Folder containing YOLO label .txt files
out_dir = ""  # Output folder for shapefiles

# === Constants ===
IMG_WIDTH =640      #  CHANGE HERE: Correct if labels were made on 1280x1280 images
IMG_HEIGHT = 640     # CHANGE HERE: Correct if labels were made on 1280x1280 images
crs_code = ""  # Coordinate reference system (example: GDA94 / MGA Zone 56 OR_ EPSG:28356)

# === Class ID to Name Mapping for Roof types and Roof Material types ===
class_map = {
    "0": "",
    "1": "",
    "2": "",
    "3": ""
}

# === Create Output Directory if It Doesn't Exist ===
os.makedirs(out_dir, exist_ok=True)

# === Function to Read PGW (World File) ===
def read_world_file(pgw_path):
    with open(pgw_path, "r") as f:
        lines = f.read().strip().split()
    A = float(lines[0])  # pixel width
    D = float(lines[1])  # rotation about y-axis
    B = float(lines[2])  # rotation about x-axis
    E = float(lines[3])  # pixel height (usually negative)
    C = float(lines[4])  # X center coordinate of top-left pixel
    F = float(lines[5])  # Y center coordinate of top-left pixel
    return A, B, D, E, C, F

# === Main Loop: Process Each PGW and Matching Label File ===
for pgw_file in os.listdir(pgw_dir):
    if not pgw_file.lower().endswith(".pgw"):
        continue

    pgw_basename = os.path.splitext(pgw_file)[0]

    # Find matching label file
    found_label = None
    for lf in os.listdir(labels_dir):
        if pgw_basename in lf and lf.lower().endswith(".txt"):
            found_label = lf
            break

    if not found_label:
        print(f"⚠️ No matching label file found for {pgw_file}")
        continue

    # Get the label file base name (without .txt extension)
    label_basename = os.path.splitext(found_label)[0]

    # Read PGW transformation
    pgw_path = os.path.join(pgw_dir, pgw_file)
    A, B, D, E, C, F = read_world_file(pgw_path)

    # Read label file
    label_path = os.path.join(labels_dir, found_label)
    all_polygons = []
    all_classes = []

    with open(label_path, "r") as lf:
        for line in lf:
            vals = line.strip().split()
            if len(vals) < 3:
                continue

            class_id = vals[0]
            coords = [float(x) for x in vals[1:]]
            points_normalized = list(zip(coords[0::2], coords[1::2]))

            polygon_pts = []
            for (nx, ny) in points_normalized:
                # Scale normalized (0–1) coordinates to 640x640 pixel space
                px = nx * IMG_WIDTH
                py = ny * IMG_HEIGHT

                # Apply affine transformation to map coordinates
                mapX = C + A * px + B * py
                mapY = F + D * px + E * py
                polygon_pts.append((mapX, mapY))

            polygon_geom = Polygon(polygon_pts)

            # OPTIONAL: Fix invalid polygons
            if not polygon_geom.is_valid:
                polygon_geom = polygon_geom.buffer(0)

            all_polygons.append(polygon_geom)
            all_classes.append(class_id)

    # Map Class IDs to Names
    all_class_names = [class_map.get(cid, "Unclassified") for cid in all_classes]

    # Build GeoDataFrame
    gdf = gpd.GeoDataFrame(
        {
            "class_id": all_classes,
            "class_name": all_class_names,
            "geometry": all_polygons
        },
        crs=crs_code
    )

    # Save Shapefile using label basename
    out_shp_path = os.path.join(out_dir, f"{label_basename}.shp")
    gdf.to_file(out_shp_path)
    print(f"✅ Created Shapefile: {out_shp_path}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.6 MB/s eta 0:00:00


**Count the numbers .shp files**

In [ ]:
import os

# Set your folder path where shapefiles are located
search_folder = ""  # <-- Change this

# Initialize counter
shp_file_count = 0

# Walk through all subdirectories
for root, dirs, files in os.walk(search_folder):
    for file in files:
        if file.lower().endswith('.shp'):
            shp_file_count += 1

print(f"Total number of .shp files: {shp_file_count}")

**Merge All .shp files into One for GIS**

In [ ]:
import geopandas as gpd
import os
import pandas as pd

# 1. Set your input folder where all the shapefiles are located
input_folder = ""

# 2. Set your output folder where you want to save the combined shapefile
output_folder = ""
output_filename = "Roof_Type_OR_Materials_combined_shapefile.shp"  # Output shapefile name

# 3. Make sure output folder exists
os.makedirs(output_folder, exist_ok=True)

# 4. List all .shp files
shapefile_list = []
for root, dirs, files in os.walk(input_folder):
    for file in files:
        if file.lower().endswith(".shp"):
            shapefile_list.append(os.path.join(root, file))

# 5. Read and combine all shapefiles
gdf_list = [gpd.read_file(shp) for shp in shapefile_list]

# 6. Concatenate into one GeoDataFrame
combined_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

# 7. Save the combined shapefile to the output folder
output_path = os.path.join(output_folder, output_filename)
combined_gdf.to_file(output_path)

print(f"✅ Successfully combined {len(shapefile_list)} shapefiles into {output_path}")